# Snomed2Vec-00.

## Generator Model - Creacion de los Modelos de Espacio Vectorial-  
## A new Approach Snomed-CT with word embedding - Word2Vec
### Introducción:
**Autor:** Ignacio Martinez Soriano.<br>
**Coautor:** Juan Luis Castro Peña.<br>
**Fecha:** 24/01/2019.<br>


### Introducción:
Creación de un nuevo enfoque **Snomed2Vec**:<br>
Aplicamos "Word Embedding" sobre las descripciones de los términos de la Ontología Clínica **Snomed-CT**. Creamos un espacio vectorial, formado por los vectores de las palabras que forman parte en cada descripción de un Concepto clínico de Snomed-CT.<br>
Generamos varios modelos vectoriales, basados en grandes conjuntos de datos:<br>
* sobre un dominio de lenguaje local: (Informes de Alta de Urgencias, Hospital "Rafael Méndez", Años 2009-2016 + las descripciones de los términos de los conceptos de Snomed-CT.
* Otro sobre un dominio general: ((Artículos de wikipedia en español descarga enero 2019 + Las descripciones de los términos de los conceptos de Snomed-CT.

Además implementamos una version de **Most_Similar()** para obtener los Conceptos similares de  **Snomed-CT** y que nos indique el tipo de concepto clínico que ha encontrado, y la asociacion con otros conceptos.<br>

* Por ejemplo, para un hallazgo clinico = diagnóstico, podemos pedirle, los procedimientos mas similares, encontrados en el espacio vectorial.

### Datos de Entrada:

1. Los Informes de Alta de Urgencias, /DataWork/**Corpus-03-tokens.txt**.(Datos con un concepto por linea, todos sus sinónimos y su jerarquia).
1. Los articulos de la  wikipedia en Español, descargada en Enero de 2019. y Extraidos los artículos con WikiExtractor. https://github.com/attardi/wikiextractor
1. Los conceptos de Snomed-CT, /DataWork/**Snomed-CorpusTerm-Jer.txt**. (Id-Concepto, jerarquia y descripcion) por linea.

Todos los textos, están normalizados, se han cambiado a minúsculas y se han eliminado los acentos.

## Productos generados:


## 1. Generación de los Modelos de Espacio Vectorial:

Para la generación de los modelos utilizamos **Word2Vec**.<br>

1. Modelo Dominio local, Word2Vec:<br>
Se genera un modelo de espacio vectorial, con **Word2Vec**, basado en los Informes de Alta de urgencias + Descripción términos de Snomed-CT:<br>
***(SkipGram, vector size 300, windows= 10, Negative=0, min=1)***

1. Modelo con un dominio General de Wikipedia en Español + las desripciones de los términos de Snomed-CT:<br>
***(SkipGram, vector size 300, windows= 10, Negative=0, min=1)***

Se puede intentar generar con otros modelos:
1. Modelo Dominio local, Doc2Vec:<br>
Se genera un modelo de espacio vectorial, con **Doc2Vec**, basado en los Informes de Alta de urgencias. Con los siguientes parámetros:

1. Modelo Dominio local, Doc2Vec:<br>
Se genera un modelo de espacio vectorial, con **Glove**, basado en los Informes de Alta de urgencias. Con los siguientes parámetros:





## 2. Estructura inicial, Espacio Vectorial final Snomed2Vec-Model:
Se propone la siguiente esctructura de la Matriz, Snomed2Vec.

Propuesta:

* Un registro para cada concepto con la siguiente estructura:<br>
    **|Id-Concept | TokensDescripcion | Jerarquia | [Vector de la Descripción]|**
    

### 2.1 Carga de librerias necesarias:

In [1]:
# Carga de librerias. Versiones en Portatil
# numpy 1.13.3
# pandas 0.20.3
# gensim 2.30
import sys
#p= r'D:\Doctorado\codigo'
p='/media/nacho/DatosHD/Doctorado/codigo'
sys.path.append(p)

#import unicodedata
import pandas as pd
import numpy as np

from gensim.models import Word2Vec

from six import iteritems, itervalues, string_types
from six.moves import xrange
from numpy import exp, log, dot, zeros, outer, random, dtype, float32 as REAL,\
    double, uint32, seterr, array, uint8, vstack, fromstring, sqrt, newaxis,\
    ndarray, empty, sum as np_sum, prod, ones, ascontiguousarray

import logging
import IPython.display as disp

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
pd.set_option('max_colwidth', 100) #para que se impriman las columnas con mayor anchura en el notebook

Using TensorFlow backend.


## Nota ---------:
**Advertencia:** Si solo se quiere generar nuevos modelos, haciendo combinaciones con los datos, salta diectamente al punto **4.1** o al **4.2** y carga el modelo inicial del los datos del Snomed-CT. **Snomed2vec-Jer-Corpus.txt**

Saltar el punto **5.3.2.** Para cargar el Dataframe final, con los vectores generados.


## 3. Creación Datos DataFrame de Snomed-CT. Estructura final

Partimos de esta estructura incial, generada en el Notebook 05-Snomed-CT-Term-Sinonimos.ipynb

### Descarga Sinónimos de  bbdd Mysql, Snomed-CT SnapShot

Una vez importados los datos  de Snomed-CT, a una bbdd MySql. Utilizando el script RF2 MySQL Import and Optimize  http://snomed.org/sctfull2mysql de la página: https://confluence.ihtsdotools.org/display/DOC/Technical+Resources En este fichero podemos encontrar todas las herramientas necesarias, para cargar y transformar los datos de las descargas oficiales de Snomed-CT.

Genero una descarga identificando el concept_Id, termino y Id_descripción utilizo las tablas de:
* snomedct.sct2_concept_snapshot_global
* snomedct.sct2_description_snapshot_global

### Conceptos general y sinónimos.
Los conceptos existentes en Snomed-CT, version española, con sus sinónimos son **(516211)**. <br>
Para obtener estos datos se filtra la tabla sct2_descrition, utilizando el typeId = '90000000000013009', este código identifica a los ***sinónimos***<br>

Se descarga utilizando la versión SnapShot con la siguiente SQL:

> **SELECT** c.id as idConcept, d.Term, d.id as id_descripcion<br>
**FROM** snomedct.sct2_concept_snapshot_global c, snomedct.sct2_description_snapshot_global d <br>
**where** (c.id = d.conceptId)<br>
	**and** c.active=1<br>
    **and** d.typeId='900000000000013009'<br>
	**and** (d.languageCode='es' **or** d.languageCode='es-ES');<br>
<br>
Esta descaga proporciona **(516211)** conceptos con sus sinónimos.<br>

**Ultima descarga Select utilizada:**

BBDD utilizada Snomed-CT_Snapshot.
Una vez descargados los datos se genera los tokens de cada descripción normalizados. y obtenemos la siguiente estructura inicial:

### 3.1. Partimos del DataSet con la descripción sin modificar de Snomed-CT:


In [2]:
# Partimos de esta estructura inicial generada seǵun el Notebook 5. Proyecto Tesis - Opcion 2
fich = '/Snomed-CT/Diccionario/Snomed-CorpusTerm-Original.txt'

f = p+fich
dfSnomedWork = pd.read_csv(f, sep='|', encoding='utf_8', header=0, dtype='object').fillna('')


In [3]:
dfSnomedWork.head()

,idConcept,Term,corpus
0,102002,hemoglobina Okaloosa,"['hemoglobina', 'Okaloosa']"
1,102002,"Hb 48 (CD7), Leu - arg","['Hb', '48', 'CD7', 'Leu', 'arg']"
2,103007,virus del fibroma de las ardillas,"['virus', 'del', 'fibroma', 'de', 'las', 'ardillas']"
3,104001,escisión de una lesión en la rótula,"['escisión', 'de', 'una', 'lesión', 'en', 'la', 'rótula']"
4,104001,escisión local de una lesión o de tejido de la rótula,"['escisión', 'local', 'de', 'una', 'lesión', 'o', 'de', 'tejido', 'de', 'la', 'rótula']"


In [4]:
print('Comprobacion de tamaño - 516211 - %d' % len(dfSnomedWork))

Comprobacion de tamaño - 516211 - 516211


### 3.2. Cargamos DataSet con la descripción Normalizada de Snomed-CT:
**Versión para generar los modelos vectoriales, con los  Informes de ALta de Urgencias y Wikipedia.**

In [44]:
# Partimos de esta estructura inicial generada seǵun el Notebook 5. Proyecto Tesis
fich = '/Snomed-CT/Diccionario/Snomed-TokenSynFinal-ok.txt'

f = p+fich
dfSnomedSyn = pd.read_csv(f, sep='|', encoding='utf_8', header=0, dtype='object').fillna('')

In [45]:
# Convertir los datos cargados en listas de texto, despues de cargar los datos.
# Cuando se guarda en csv el pandas dataframe, guarda los valores como strings y hay que convertirlos a listas
from ast import literal_eval
dfSnomedSyn['corpus_token'] = dfSnomedSyn['corpus_token'].apply(literal_eval)
dfSnomedSyn['corpus_Syn'] = dfSnomedSyn['corpus_Syn'].apply(literal_eval)

In [46]:
dfSnomedSyn.head()

,idConcept,corpus,corpus_token,corpus_Syn
0,10000006,dolor de pecho irradiado hacia otras zonas,"[pecho, irradiado, hacia, dolor, zonas]","[[dolor, pecho, irradiado, hacia, zonas]]"
1,10001000122108,observaciones de la historia social relacionadas con el habito de fumar alcohol y dieta,"[relacionadas, observaciones, dieta, habito, fumar, y, social, alcohol, historia]","[[observaciones, historia, social, relacionadas, habito, fumar, alcohol, y, dieta]]"
2,10001005,septicemia bacteriana | sepsis bacteriana,"[bacteriana, sepsis, septicemia]","[[septicemia, bacteriana], [sepsis, bacteriana]]"
3,10001941000122105,modulo para componentes del nomenclator de formulas magistrales,"[magistrales, formulas, componentes, nomenclator, modulo]","[[modulo, componentes, nomenclator, formulas, magistrales]]"
4,10002003,reseccion del fondo gastrico | fundectomia gastrica | reseccion de la cupula del estomago,"[fondo, gastrica, fundectomia, estomago, reseccion, cupula, gastrico]","[[reseccion, fondo, gastrico], [fundectomia, gastrica], [reseccion, cupula, estomago]]"


### 3.3. Creación Corpus de Trabajo Final:
Aqui tenemos un concepto con todos sus sinónimos por linea.

In [47]:
# Selecciono
cols=['idConcept','corpus']
dfSno=dfSnomedSyn[cols].copy()

In [48]:
dfSno.head()

,idConcept,corpus
0,10000006,dolor de pecho irradiado hacia otras zonas
1,10001000122108,observaciones de la historia social relacionadas con el habito de fumar alcohol y dieta
2,10001005,septicemia bacteriana | sepsis bacteriana
3,10001941000122105,modulo para componentes del nomenclator de formulas magistrales
4,10002003,reseccion del fondo gastrico | fundectomia gastrica | reseccion de la cupula del estomago


In [49]:
# Generacion del Corpus de Trabajo final: Solo el id concept con sus descripciones
fich = '/Snomed-CT/Diccionario/Snomed2vec-Corpus.txt'
f = p+fich

dfSno.to_csv(f, sep='|', encoding='utf_8', header=1, index=False)

### 3.4. Carga Datos DataFrame de Snomed-CT. Snomed2Vec-Corpus.txt

In [2]:
# Carga del Snomed-CT Corpus Snomed2Vec:
fich = '/Snomed-CT/Diccionario/Snomed2vec-Corpus.txt'

f = p+fich
dfSno = pd.read_csv(f, sep='|', encoding='utf_8', header=0, dtype='object').fillna('')

In [3]:
dfSno.head()

,idConcept,corpus
0,10000006,dolor de pecho irradiado hacia otras zonas
1,10001000122108,observaciones de la historia social relacionadas con el habito de fumar alcohol y dieta
2,10001005,septicemia bacteriana | sepsis bacteriana
3,10001941000122105,modulo para componentes del nomenclator de formulas magistrales
4,10002003,reseccion del fondo gastrico | fundectomia gastrica | reseccion de la cupula del estomago


In [6]:
print('Comprobacion de tamaño - 328384 - %d' % len(dfSno))

Comprobacion de tamaño - 328384 - 328384


## 3.5. Funciones Auxiliares:

### 3.5.1. Creacion de las jerarquias Snomed-Concept:

In [7]:
DicJerarquias = {'404684003':'Hallazgo clínico','71388002':'Procedimiento','363787002':'Entidad observable','123037004':'Estructura corporal',
            '410607006':'Organismo','105590001':'Sustancia','373873005':'Producto farmacéutico / biológico','123038009':'Espécimen','370115009':'Concepto especial',
            '900000000000441000':'Componente del modelo de SNOMED CT','78621006':'Fuerza física','272379006':'Evento','308916002':'Ambiente o localización geográfica',
            '48176007':'Contexto social','243796009':'Situación con contexto explícito','254291000':'Estadificaciones y escalas',
            '260787004':'Elemento de registro','362981000':'Calificador','419891008':'Objeto físico'}

In [8]:
# Ejemplo de un elemento del diccionario: comprobaacion)
DicJerarquias.get('419891008')

'Objeto físico'

In [9]:
# Items delDiccionario de Jerarquias de primer Nivel de Snomed-CT
# Se crea una lista solo con los items del diccionario.
idLevel = ['404684003',
 '71388002',
 '363787002',
 '123037004',
 '410607006',
 '105590001',
 '373873005',
 '123038009',
 '370115009',
 '900000000000441000',
 '78621006',
 '272379006',
 '308916002',
 '48176007',
 '243796009',
 '254291000',
 '260787004',
 '362981000',
 '419891008']

### 3.5.2. Transitive Closure:
Para poder obtener la jerarquía superior de un concepto, es necesario, utilizar "Transitive Closure", una manera de recorrer todos los ancestros de un objeto, hasta llegar al nivel superior.

En un grafo dirijido es un sistema para identificar los ancestros de cualquier nodo.

Se puede utilizar alguna de las siguientes  implementaciones:
1. https://github.com/WestCoastInformatics/SNOMED-CT-Transitive-Closure
1. https://confluence.ihtsdotools.org/display/DOCTSG/7.5.2+Transitive+closure+implementation (Doc. oficial)
1. https://doc.ihtsdo.org/en_us/tig.html?t=tsg2_test_optimizeConcept_transitiveClosure_impl
1. https://github.com/IHTSDO/snomed-publish/blob/master/doc/transitiveClosure2.pl (perl)

In [10]:
# Utilizar Tabla de relaciones Transitive Closuore
#fich = '/Snomed-CT/Snomed-Data/Diccionario/transitiveClosure.csv'
fich = '/Snomed-CT/Snomed-Data/Diccionario/Snomed-TransitiveClosure.csv'

f = p+fich
dfTrans = pd.read_csv(f, sep=',', encoding='utf_8', header=0, dtype='object').fillna('')

In [11]:
print('Tamaño de la tabla generada Transitive Closure: %d' % len(dfTrans))

6115546

### 3.6 Funcion para detectar la Jerarquia de un ConceptoID:
Comprobació del funciomnamiento correcto de la descarga generada.

In [12]:
def buscaJerarquia(conceptId):
    selJerarquia=[]
    buffList=[]
    buff0=''
    
    buffer = dfTrans.loc[dfTrans['subtype']==conceptId, ['supertype']]
    buffList = buffer['supertype'].tolist()
    print(buffList)
    print('----------')
    selJerarquia = [x for x in buffList if x in idLevel]
    print(selJerarquia)
    if selJerarquia:
        buff0=DicJerarquias.get(selJerarquia[0])
    return buff0

In [13]:
# Ejemplo de obtener l aJerarquia según el concepto que se le pase de parámetro:
# esguince 384709000
buscaJerarquia('384709000')

['118956008', '37782003', '399917004', '49755003', '138875005', '123037004', '19130008', '52988006']
----------
['123037004']


'Estructura corporal'

In [14]:
# Ejemplo de obtener la Jerarquia según el concepto que se le pase de parámetro:
# septicemia bacteriana
buscaJerarquia('10001005')

['138875005', '238149007', '128139000', '404684003', '91302008', '87628006', '64572001', '40733004']
----------
['404684003']


'Hallazgo clínico'

In [ ]:
# Tamaño del corpus de Snomed Conceptos válidos.
print(dfSno.index)

In [29]:
dfTrans.index.rename('idx', inplace=True)
#data.set_index("idx", inplace=True)

In [32]:
dfTrans.index

RangeIndex(start=0, stop=6115546, step=1, name='idx')

### 3.6.1. Funcion para detectar la Jerarquia de un códigoID Snomed en un DataFrame:
Funcion utilizadavpara generar el corpus final con las Jerarquias.

In [18]:
def buscaJerDF(row):
    
    selJerarquia=[]
    buffList=[]
    buff0=''
    concept = row.idConcept
    
    buffer = dfTrans.loc[dfTrans['subtype']==concept, ['supertype']]
    buffList = buffer['supertype'].tolist()
    
    selJerarquia = [x for x in buffList if x in idLevel]
    if selJerarquia:
        buff0=DicJerarquias.get(selJerarquia[0])
    print('%s - %s' % (concept, buff0))
    return buff0

In [38]:
dfSno['jerarquia'] = dfSno.loc[0:3].apply(buscaJerDF, axis=1)

100000000 - 
10000006 - Hallazgo clínico
1000004 - 
100009004 - 


### 3.7. Proceso para generar el diccionario con los conceptos y sus jerarquias:
**Atencion** ----------- Este proceso dura **48 horas** --------------: **Atencion** ----------------------

In [22]:
# Proceso para generar el diccionario con los conceptos y sus jerarquias-----
#dfSno['jerarquia'] = dfSno.apply(buscaJerDF, axis=1)

### 3.8. Actualizar Dataframe dfSnomed-CT para añadir la Jerarquia:

In [20]:
# Actualizar dfSnomed para añadir la Jerarquia:
dfSno.head()

,idConcept,corpus,jerarquia
0,10000006,dolor de pecho irradiado hacia otras zonas,Hallazgo clínico
1,10001000122108,observaciones de la historia social relacionadas con el habito de fumar alcohol y dieta,Entidad observable
2,10001005,septicemia bacteriana | sepsis bacteriana,Hallazgo clínico
3,10001941000122105,modulo para componentes del nomenclator de formulas magistrales,
4,10002003,reseccion del fondo gastrico | fundectomia gastrica | reseccion de la cupula del estomago,Procedimiento


In [21]:
# Generacion del Corpus de Trabajo final:
fich = '/Snomed-CT/Diccionario/Snomed2vec-JerarquiaCorpus.txt'
f = p+fich

dfSno.to_csv(f, sep='|', encoding='utf_8', header=1, index=False)

In [23]:
len(dfSno)

328384

### 3.9. Carga de Datos Snomed IdConcepto y Jerarquias:

In [ ]:
# Carga del Snomed-CT.
fich = '/Snomed-CT/Diccionario/Snomed2vec-JerarquiaCorpus.txt'

f = p+fich
dfSnoJer = pd.read_csv(f, sep='|', encoding='utf_8', header=0, dtype='object').fillna('')

In [ ]:
print('Comprobacion de tamaño - 328384 - %d' % len(dfSnoJer))

### 3.9.1. Unificar en un solo Corpus con todos los datos:

In [ ]:
cols=['idConcept','jerarquia']
dfSnoJerCorpus = pd.merge(dfSnoJer[cols], dfSnomedSyn, on='idConcept')

In [ ]:
dfSnoJerCorpus.head(5)

### 3.9.2. Guardado del Corpus final de Trabajo:
Corpus final **Snomed2vec-Jer-Corpus.txt**

In [ ]:
# Generacion del Corpus de Trabajo final:
fich = '/Snomed-CT/Diccionario/Snomed2vec-Jer-Corpus.txt'
f = p+fich

dfSnoJerCorpus.to_csv(f, sep='|', encoding='utf_8', header=1, index=False)

In [142]:
cols=['idConcept','jerarquia','corpus_Syn']
dfSnoWork=dfSnoJerCorpus[cols].copy()

In [143]:
dfSnoWork.head(4)

,idConcept,jerarquia,corpus_Syn
0,10000006,Hallazgo clínico,"[[dolor, pecho, irradiado, hacia, zonas]]"
1,10001000122108,Entidad observable,"[[observaciones, historia, social, relacionadas, habito, fumar, alcohol, y, dieta]]"
2,10001005,Hallazgo clínico,"[[septicemia, bacteriana], [sepsis, bacteriana]]"
3,10001941000122105,,"[[modulo, componentes, nomenclator, formulas, magistrales]]"


In [144]:
print('Comprobacion de tamaño - 328384 - %d' % len(dfSnoWork))

Comprobacion de tamaño - 328384 - 328384


In [145]:
# Generacion del Corpus de Trabajo final: (Sin el campo corpus_token)
fich = '/Snomed-CT/Diccionario/Snomed2vec-Jer-work.txt'
f = p+fich

dfSnoWork.to_csv(f, sep='|', encoding='utf_8', header=1, index=False)

## 4 - Creación del modelo Word2Vec:
### Utilizando los Informes de Alta de Urgencias + las Descripciones de los términos de Snomed-CT
**Prueba de Modelo 01** (Datos obtenidos de los Informes de Alta:<br>
*SkipGram, vector size 300, windows= 10, Negative=0, min=1*

### 4.1 Carga de Datos de la estructura de Snomed-CT final:(Opcion 1.)
**Snomed-CorpusTerm-Jer.txt** <br>
Contiene una linea por cada descripcion del concepto (idConcept, cospusTerm, jerarquia)
Creamos el dataframe **dfSnoWork**

In [2]:
# Carga de Datos para el Trabajo final:
fich = '/DataWork/Snomed-CorpusTerm-Jer.txt'
f = p+fich

dfSnoWork = pd.read_csv(f, sep='|', encoding='utf_8', header=0, dtype='object').fillna('')

In [3]:
# Convertir los datos cargados en listas de texto, despues de cargar los datos.
from ast import literal_eval
dfSnoWork['corpusTerm'] = dfSnoWork['corpusTerm'].apply(literal_eval)
dfSnoWork.head()

,idConcept,corpusTerm,jerarquia
0,102002,"[hemoglobina, okaloosa]",Sustancia
1,102002,"[hb, 48, cd7, leu, arg]",Sustancia
2,103007,"[virus, fibroma, ardillas]",Organismo
3,104001,"[escision, lesion, rotula]",Procedimiento
4,104001,"[escision, local, lesion, o, tejido, rotula]",Procedimiento


In [4]:
print('Tamaño del dataframe Opcion.2. = 516211, para la gestion de los códigos de Snomed: %d' % len(dfSnoWork))

Tamaño del dataframe Opcion.2. = 516211, para la gestion de los códigos de Snomed: 516211


### 4.3 Generación del Modelo 01 (Word2Vec)- Modelo Espacio Vectorial Opcion 1.
**Modelo generado de los Informes de Alta del Hospital** (opcion Modelo 1)

Modelo: **SkipGram, vector size 300, windows= 3, Negative=0, min=1** utilizando  la  libreria gensim sobre los Informes de ALta del Hospital. 

In [8]:
# Carga de un modelo Skip-Gram, con tamaño vector de 300
from gensim.models.keyedvectors import KeyedVectors
#fich = '/Models/w2v-SK-s300-w5.txt'
fich = '/Models/w2v-SK-s300-w5-min2.txt'
#fich = '/Models/w2v-SK-s50-w15-Neg10.txt'
f = p+fich
w2v_InfAltaSK = KeyedVectors.load_word2vec_format(f, binary=False)

2018-09-18 18:07:31,564 : INFO : loading projection weights from /media/nacho/DatosHD/Doctorado/codigo/Models/w2v-SK-s300-w5-min2.txt
2018-09-18 18:07:47,558 : INFO : loaded (98103, 300) matrix from /media/nacho/DatosHD/Doctorado/codigo/Models/w2v-SK-s300-w5-min2.txt


### 4.3 - Carga del modelo Word2Vec: Creado Sobre las descripciones de Snomed-CT (opcion2)
SkipGram, vector size 300, windows= 3, Negative=0, min=1

In [5]:
# Carga de un modelo Skip-Gram, con tamaño vector de 300
from gensim.models.keyedvectors import KeyedVectors
#fich = '/Models/w2vSnomed-SK-s300-w3.txt'
fich = '/Models/w2v-SK-s300-w5-m2-Total-Key.txt'
f = p+fich
w2v_SnomedSK = KeyedVectors.load_word2vec_format(f, binary=False)

2018-10-04 18:26:33,163 : INFO : loading projection weights from /media/nacho/DatosHD/Doctorado/codigo/Models/w2v-SK-s300-w5-m2-Key.txt
2018-10-04 18:27:19,962 : INFO : loaded (294700, 300) matrix from /media/nacho/DatosHD/Doctorado/codigo/Models/w2v-SK-s300-w5-m2-Key.txt


### 4.4 - Carga del modelo Word2Vec: Creado Sobre Wikipedia Español  (opcion3):
CBoW, vector size 300, windows= 3, Negative=0, min=1

In [6]:
# Carga de un modelo Skip-Gram, con tamaño vector de 300
from gensim.models.keyedvectors import KeyedVectors
fich = '/Models/SBW-vectors-300-min5.txt'
f = p+fich
w2v_wikiCBoW= KeyedVectors.load_word2vec_format(f, binary=False)

2018-09-30 09:26:33,385 : INFO : loading projection weights from /media/nacho/DatosHD/Doctorado/codigo/Models/SBW-vectors-300-min5.txt
2018-09-30 09:29:24,958 : INFO : loaded (1000653, 300) matrix from /media/nacho/DatosHD/Doctorado/codigo/Models/SBW-vectors-300-min5.txt


## 5. Creación de los vectores para las descripciones de Snomed-CT:

Una vez elegido el Modelo, se utilizará,para generar los vectores de cada descripción de Snomed-CT.

El vector generado para la descripción **[d]** está formado  por:
* $ Descripcion (d) = (w_1,w_2,...,w_n), V[d] = \sum_{i=1}^{n}v[x_{i}]=v[x_{1}]+v[x_{2}]+...+v[x_{n}] $



### 5.1. Crear Vector para cada token de la frase.
Generación de los vectores de los tokens de cada frase.<br>

* Funcion: **vectorTotal(listGramas, size):**
    * Entrada: **listGramas**, lista de n-gramas generados, **size**, tamaño del vector, según modelo W2V.
    * Salida: Vectores asociados a cada frase, si es 1-grama -> Salida Vector(token).<br>
      Si la frase tiene **n** gramas -> <br>
      Salida:$$(V[p]=v[tk_{1}]+v[tk_{2}]+...+v[tk_{n}])$$

In [8]:
# Funcion: lista de Frase, con ngramas en sublistas - > Devuelve el Vector segun el modelo cargado

# Entrada: (lstgramas=[['fiebre','amarilla'],['tos']], size=300)
# Salida: Vector para cada item de de la lista

def vectorTotal(lstgrams,size):
    output=[]
    for item in lstgrams:
        buffer=np.zeros((size,), dtype=np.float32)
        for w in item:
            try:
                buffer = buffer + w2v_InfAltaSK.word_vec(w)
            except:
                # Si el token no está en el vocabulario del Modelo, se descarta todos los ngramas
                buffer = buffer + np.zeros((size,), dtype=np.float32)
                #break
        output.append((item, buffer))
    return output

In [7]:
# Funcion: lista de Frase, con ngramas en sublistas - > Devuelve el Vector segun el modelo cargado

# Entrada: (lstgramas=['fiebre','amarilla'], model, size=300)
# Salida: Vector para cada item de de la lista

def vectorTotalOpci2(lstgrams,model,size):
    output=[]
    buffer=np.zeros((size,), dtype=np.float32)
    for w in lstgrams:
        try:
            buffer = buffer + model.word_vec(w)
        except:
            # Si el token no está en el vocabulario del Modelo, se genera un vector con 0.0
            buffer = buffer + np.zeros((size,), dtype=np.float32)

    output.append(buffer)
    return output

### 5.2. Creación vectores finales todo el Snomed-CT.
Se genera un vector para cada descripción de cada concepto y se genera la matriz de conceptos.
Dos definiciones de la funcion **creaVecSnomed**

In [10]:
def creaVecSnomed(row):
    #output=[]
    #desc=row.corpus_Syn.tolist()
    desc=row.corpus_Syn
    cod = row.idConcept
    #print(desc[0])
    logging.info("PROGRESS: Id-Concept #%i ", int(cod) )
    #return vectorTotal(desc[0], 300)
    return vectorTotal(desc, 300)

In [11]:
def creaVecSnomedOpci2(row):

    desc=row.corpusTerm
    cod = row.idConcept
    #logging.info("PROGRESS - Opcion 2.: Id-Concept #%i ", int(cod) )
    output=[]
    buffer=np.zeros((300,), dtype=np.float32)
    for w in desc:
        try:
            buffer = buffer + w2v_InfAltaSK.word_vec(w)
        except:   
            buffer = buffer + np.zeros((300,), dtype=np.float32)

    output.append(buffer)   
    
    return output

In [10]:
def creaVecSnomedOpci3(row):

    desc=row.corpus
    cod = row.idConcept
    
    output=[]
    buffer=np.zeros((300,), dtype=np.float32)
    for w in desc:
        try:
            buffer = buffer + w2v_wikiCBoW.word_vec(w)
        except:   
            buffer = buffer + np.zeros((300,), dtype=np.float32)

    output.append(buffer)   
    
    return output

In [11]:
def creaVecSnomedOpci4(row):

    desc=row.corpusTerm
    cod = row.idConcept
    
    output=[]
    buffer=np.zeros((300,), dtype=np.float32)
    for w in desc:
        try:
            buffer = buffer + w2v_SnomedSK.word_vec(w)
        except:   
            buffer = buffer + np.zeros((300,), dtype=np.float32)

    output.append(buffer)   
    
    return output

### Creación del Vector Opcion 01.
Se genera los vectores utilizando los tokens de cada sinónimo, utilizamos el campo **corpus_Sym**

In [ ]:
# Creacion Modelo Opcion 01--------------------------------------------------------:
#cols=['idConcept', 'jerarquia', 'corpus_Syn']
dfSnoWork['vecSnom']=dfSnoWork.apply(creaVecSnomed, axis=1)

In [10]:
print('Tamaño del dataframe para la gestion de los códigos de Snomed: %d' % len(dfSnoWork))

Tamaño del dataframe para la gestion de los códigos de Snomed: 328384


In [11]:
dfSnoWork.head()

,idConcept,jerarquia,corpus_Syn,vecSnom
0,10000006,Hallazgo clínico,"[[dolor, pecho, irradiado, hacia, zonas]]","[([dolor, pecho, irradiado, hacia, zonas], [-1.49022, 2.47072, -0.73461, -0.84359, -0.613186, -1..."
1,10001000122108,Entidad observable,"[[observaciones, historia, social, relacionadas, habito, fumar, alcohol, y, dieta]]","[([observaciones, historia, social, relacionadas, habito, fumar, alcohol, y, dieta], [3.99719, -..."
2,10001005,Hallazgo clínico,"[[septicemia, bacteriana], [sepsis, bacteriana]]","[([septicemia, bacteriana], [-0.116421, 0.750128, 0.433537, -0.776152, -0.166538, -0.133608, 0.0..."
3,10001941000122105,,"[[modulo, componentes, nomenclator, formulas, magistrales]]","[([modulo, componentes, nomenclator, formulas, magistrales], [0.095383, 0.155171, -0.002558, -0...."
4,10002003,Procedimiento,"[[reseccion, fondo, gastrico], [fundectomia, gastrica], [reseccion, cupula, estomago]]","[([reseccion, fondo, gastrico], [1.15609, 0.55614, -0.024078, -0.230703, -0.447392, -0.950934, -..."


### 5.2.1. Creacion delModelo final, para la opción 2. Una linea por cada descripcion y concepto.

In [8]:
dfSnoWork.loc[0:10]

,idConcept,corpusTerm,jerarquia
0,102002,"[hemoglobina, okaloosa]",Sustancia
1,102002,"[hb, 48, cd7, leu, arg]",Sustancia
2,103007,"[virus, fibroma, ardillas]",Organismo
3,104001,"[escision, lesion, rotula]",Procedimiento
4,104001,"[escision, local, lesion, o, tejido, rotula]",Procedimiento
5,104001,"[escision, lesion, rotula]",Procedimiento
6,104001,"[reseccion, lesion, rotula]",Procedimiento
7,104001,"[reseccion, lesion, rotula]",Procedimiento
8,106004,"[estructura, region, carpiana, posterior]",Estructura corporal
9,106004,"[region, carpiana, posterior]",Estructura corporal


In [9]:
# Creacion Modelo Opcion 02:
dfSnoWork['vecSnom']=dfSnoWork.apply(creaVecSnomedOpci2, axis=1)

In [10]:
dfSnoWork.head(10)

,idConcept,corpusTerm,jerarquia,vecSnom
0,102002,"[hemoglobina, okaloosa]",Sustancia,"[[-0.001869, -0.106874, -0.030565, -0.029062, -0.006448, 0.024021, -0.046014, -0.031549, -0.0867..."
1,102002,"[hb, 48, cd7, leu, arg]",Sustancia,"[[0.132827, -0.037834, -0.160544, -0.15251, 0.075627, -0.151947, -0.02046, 0.102717, 0.015968, -..."
2,103007,"[virus, fibroma, ardillas]",Organismo,"[[0.159795, -0.12543, 0.013634, 0.064749, 0.198714, -0.031992, -0.009603, -0.013956, -0.073225, ..."
3,104001,"[escision, lesion, rotula]",Procedimiento,"[[0.226914, -0.014378, 0.117642, -0.132778, -0.020993, 0.016509, -0.036435, -0.101064, -0.041278..."
4,104001,"[escision, local, lesion, o, tejido, rotula]",Procedimiento,"[[0.160725, -0.313286, 0.246844, -0.291712, -0.074453, -0.062273, -0.136308, -0.108905, 0.086968..."
5,104001,"[escision, lesion, rotula]",Procedimiento,"[[0.226914, -0.014378, 0.117642, -0.132778, -0.020993, 0.016509, -0.036435, -0.101064, -0.041278..."
6,104001,"[reseccion, lesion, rotula]",Procedimiento,"[[0.226914, -0.014378, 0.117642, -0.132778, -0.020993, 0.016509, -0.036435, -0.101064, -0.041278..."
7,104001,"[reseccion, lesion, rotula]",Procedimiento,"[[0.226914, -0.014378, 0.117642, -0.132778, -0.020993, 0.016509, -0.036435, -0.101064, -0.041278..."
8,106004,"[estructura, region, carpiana, posterior]",Estructura corporal,"[[0.262904, -0.125199, 0.076631, -0.148731, -0.076016, 0.015141, -0.185137, 0.098186, 0.236993, ..."
9,106004,"[region, carpiana, posterior]",Estructura corporal,"[[0.186643, -0.06458, 0.055436, -0.164874, -0.067108, -0.092796, -0.080163, -0.036678, 0.176424,..."


### 5.2.2. Creación modelo final Vectorizado, Opción 3: Modelo Wikipedia.

In [19]:
# Creacion Modelo Opcion 03:
dfSnomedWork['vecSnom']=dfSnomedWork.apply(creaVecSnomedOpci3, axis=1)

In [18]:
# Convertir los datos cargados en listas de texto, despues de cargar los datos para el corpus.
from ast import literal_eval
dfSnomedWork['corpus'] = dfSnomedWork['corpus'].apply(literal_eval)
dfSnomedWork.head()

,idConcept,Term,corpus,vecSnom
0,102002,hemoglobina Okaloosa,"[hemoglobina, Okaloosa]","[-0.573977, -0.34192, -0.704767, -0.253131, 0.195316, -0.335175, 0.175234, 0.386643, 0.920162, -..."
1,102002,"Hb 48 (CD7), Leu - arg","[Hb, 48, CD7, Leu, arg]","[0.050742, -0.16843, -0.330733, -0.561051, -0.125784, -0.368998, 0.00823702, 0.265023, 0.299335,..."
2,103007,virus del fibroma de las ardillas,"[virus, del, fibroma, de, las, ardillas]","[-0.456331, -0.243364, -1.54545, -0.235181, 0.104182, -0.993441, 0.285053, 0.745215, 1.10138, -0..."
3,104001,escisión de una lesión en la rótula,"[escisión, de, una, lesión, en, la, rótula]","[-0.327545, -0.159125, -0.953058, -0.410016, 0.268365, -0.940003, 0.466508, 0.995458, 0.888104, ..."
4,104001,escisión local de una lesión o de tejido de la rótula,"[escisión, local, de, una, lesión, o, de, tejido, de, la, rótula]","[-0.78032, -0.45486, -1.63683, -0.442614, 0.185314, -1.32256, 0.330236, 1.52869, 1.71968, 0.0439..."


In [8]:
dfSnoWork.head()

,idConcept,corpusTerm,jerarquia
0,102002,"[hemoglobina, okaloosa]",Sustancia
1,102002,"[hb, 48, cd7, leu, arg]",Sustancia
2,103007,"[virus, fibroma, ardillas]",Organismo
3,104001,"[escision, lesion, rotula]",Procedimiento
4,104001,"[escision, local, lesion, o, tejido, rotula]",Procedimiento


### 5.2.3. Creación modelo final Vectorizado, Opción 4: Modelo Total (InfAlta + Snomed-CT)

In [13]:
# Creacion Modelo Opcion 04:
dfSnoWork['vecSnom']=dfSnoWork.apply(creaVecSnomedOpci4, axis=1)

In [16]:
dfSnoWork.head()

,idConcept,corpusTerm,jerarquia,vecSnom
0,102002,"[hemoglobina, okaloosa]",Sustancia,"[[-0.660218, 0.112245, 0.067039, 0.441094, -0.50456, -0.724666, -0.107986, 0.313558, 0.198024, 1..."
1,102002,"[hb, 48, cd7, leu, arg]",Sustancia,"[[-0.317944, 1.87942, 1.76004, 1.72051, 0.75724, -1.13105, -0.499457, -1.84984, -0.570444, -0.63..."
2,103007,"[virus, fibroma, ardillas]",Organismo,"[[-0.14364, -0.006139, 0.080456, 0.936581, -1.16385, -0.552844, 0.081489, -0.584596, -0.185695, ..."
3,104001,"[escision, lesion, rotula]",Procedimiento,"[[-1.0588, -0.371483, -1.84789, 1.65618, -0.235278, 0.101576, -1.08105, 0.863104, -0.374193, 1.0..."
4,104001,"[escision, local, lesion, o, tejido, rotula]",Procedimiento,"[[-0.178396, -0.865999, -2.62099, 3.59125, -1.68036, -0.115811, -1.95361, 0.583631, -1.50294, 1...."


In [15]:
dfSnoWork[dfSnoWork['corpusTerm']=='infarto']

,idConcept,corpusTerm,jerarquia,vecSnom


In [25]:
dfSnomedWork[dfSnomedWork['Term']=='infarto']

,idConcept,Term,corpus,vecSnom
81776,55641003,infarto,[infarto],"[-0.007802, -0.032055, 0.055065, -0.021404, 0.037289, 0.013248, 0.005701, -0.003707, -0.02746, -..."


In [ ]:
w2v_wikiCBoW.word_vec('infarto')

** 5.2.1.1. Hacer unas adptaciones, para crear correctamente la columna vecSnom -> numpy.array(300,)**

In [17]:
def cambiaNumpy(row):
    vocab = row[0]
    #print(vocab)
    voc = np.array(vocab, dtype=REAL)
    return voc

In [22]:
# Opcion 3. Actualizr vecSnom column
dfSnomedWork['vecSnom'] = dfSnomedWork['vecSnom'].apply(cambiaNumpy)

In [18]:
# Opcion 4. Actualizr vecSnom column
dfSnoWork['vecSnom'] = dfSnoWork['vecSnom'].apply(cambiaNumpy)

In [19]:
dfSnoWork.head()

,idConcept,corpusTerm,jerarquia,vecSnom
0,102002,"[hemoglobina, okaloosa]",Sustancia,"[-0.660218, 0.112245, 0.067039, 0.441094, -0.50456, -0.724666, -0.107986, 0.313558, 0.198024, 1...."
1,102002,"[hb, 48, cd7, leu, arg]",Sustancia,"[-0.317944, 1.87942, 1.76004, 1.72051, 0.75724, -1.13105, -0.499457, -1.84984, -0.570444, -0.630..."
2,103007,"[virus, fibroma, ardillas]",Organismo,"[-0.14364, -0.006139, 0.080456, 0.936581, -1.16385, -0.552844, 0.081489, -0.584596, -0.185695, 0..."
3,104001,"[escision, lesion, rotula]",Procedimiento,"[-1.0588, -0.371483, -1.84789, 1.65618, -0.235278, 0.101576, -1.08105, 0.863104, -0.374193, 1.04..."
4,104001,"[escision, local, lesion, o, tejido, rotula]",Procedimiento,"[-0.178396, -0.865999, -2.62099, 3.59125, -1.68036, -0.115811, -1.95361, 0.583631, -1.50294, 1.5..."


In [14]:
dfSnoWork['vecSnom'] = dfSnoWork['vecSnom'].apply(cambiaNumpy)

In [22]:
dfSnoWork.head(10)

,idConcept,corpusTerm,jerarquia,vecSnom
0,102002,"[hemoglobina, okaloosa]",Sustancia,"[-0.660218, 0.112245, 0.067039, 0.441094, -0.50456, -0.724666, -0.107986, 0.313558, 0.198024, 1...."
1,102002,"[hb, 48, cd7, leu, arg]",Sustancia,"[-0.317944, 1.87942, 1.76004, 1.72051, 0.75724, -1.13105, -0.499457, -1.84984, -0.570444, -0.630..."
2,103007,"[virus, fibroma, ardillas]",Organismo,"[-0.14364, -0.006139, 0.080456, 0.936581, -1.16385, -0.552844, 0.081489, -0.584596, -0.185695, 0..."
3,104001,"[escision, lesion, rotula]",Procedimiento,"[-1.0588, -0.371483, -1.84789, 1.65618, -0.235278, 0.101576, -1.08105, 0.863104, -0.374193, 1.04..."
4,104001,"[escision, local, lesion, o, tejido, rotula]",Procedimiento,"[-0.178396, -0.865999, -2.62099, 3.59125, -1.68036, -0.115811, -1.95361, 0.583631, -1.50294, 1.5..."
5,104001,"[escision, lesion, rotula]",Procedimiento,"[-1.0588, -0.371483, -1.84789, 1.65618, -0.235278, 0.101576, -1.08105, 0.863104, -0.374193, 1.04..."
6,104001,"[reseccion, lesion, rotula]",Procedimiento,"[-0.982033, -0.414664, -1.92898, 1.4699, -0.133419, 0.178727, -1.12998, 0.707548, -0.191675, 0.8..."
7,104001,"[reseccion, lesion, rotula]",Procedimiento,"[-0.982033, -0.414664, -1.92898, 1.4699, -0.133419, 0.178727, -1.12998, 0.707548, -0.191675, 0.8..."
8,106004,"[estructura, region, carpiana, posterior]",Estructura corporal,"[1.32918, -2.10342, -0.835354, 1.25112, -1.40627, 0.163855, -0.394745, -0.163158, -1.36843, -0.0..."
9,106004,"[region, carpiana, posterior]",Estructura corporal,"[0.891201, -1.78964, -1.04128, 1.19131, -0.912138, 0.298044, -0.25176, -0.086428, -1.10025, -0.3..."


In [ ]:
# Ejemplo para elminiar una columna en Pandas:
SnomedWork.drop('vecSnom', axis=1, inplace=True)

In [ ]:
# Ejemplo de como renombrar una columan en pandas:
SnomedWork.rename(columns={'vecSnom1': 'vecSnom'}, inplace=True)

### 5.3. Grabamos el segundo modelo de Snomed2Vec. Modelo generado con los informes de ALta. Opcion.2:
**Snomed2Vec-SK300-IA-Op2.txt**
Se guarda el fichero en formato numpy. Se utiliza esta versión para almacenar correctamente los vectores:

** 5.3.1. grabar el datarame final.**

In [16]:
# Prueba con fichero numpy. Guardar Se utiliza esta versión para almacenar correctamente los vectores:
fich = '/Snomed-CT/Diccionario/Snomed2Vec-COBW300-Wiki-Op2.npy'
#fich = '/Snomed-CT/Diccionario/Snomed2Vec-SK300-IA-Op2-ok.npy'
f = p+fich
cols=['idConcept','jerarquia','corpusTerm','vecSnom']
num_df = dfSnoWork[cols]

with open(f,'wb') as np_file:
    np.save(np_file, num_df)

** 5.3.2. Cargar el modelo previamente generado en formato numpy.**<br>
fichero *Snomed2Vec-SK300-IA-Op2.npy*

In [2]:
# Prueba de Carga numpy ------------------------
fich = '/Snomed-CT/Diccionario/Snomed2Vec-SK300-IA-Op2.npy'
f = p+fich
cols=['idConcept','jerarquia','corpusTerm','vecSnom']

with open(f,'rb') as np_file:
    data = np.load(np_file)
SnomedWork = pd.DataFrame(data, columns=cols)

** 5.3.3. grabar el datarame final - Descripciones Originales.****<br>
fichero *Snomed2Vec-SK300-Wiki-Original-Op3.npy*

In [27]:
# Prueba con fichero numpy. Guardar Se utiliza esta versión para almacenar correctamente los vectores:
fich = '/Snomed-CT/Diccionario/Snomed2Vec-COBW300-Wiki-Op3.npy'
f = p+fich
cols=['idConcept','Term','vecSnom']
num_df = dfSnomedWork[cols]

with open(f,'wb') as np_file:
    np.save(np_file, num_df)

** 5.3.4. grabar el datarame final - Descripciones Normalizadas.Modelo (InfAltas+Snomed-CT)**<br>
fichero Salida: *Snomed2Vec-SK300-InfAlt-Snomed-Op4.npy*

In [24]:
# Prueba con fichero numpy. Guardar Se utiliza esta versión para almacenar correctamente los vectores:
fich = '/Snomed-CT/Diccionario/Snomed2Vec-SK300-InfAlt-Snomed-Op4.npy'
f = p+fich
cols=['idConcept','jerarquia','corpusTerm','vecSnom']
num_df = dfSnoWork[cols]

with open(f,'wb') as np_file:
    np.save(np_file, num_df)

In [23]:
# Comprobacion del dataframe:
dfSnoWork.head()

,idConcept,corpusTerm,jerarquia,vecSnom
0,102002,"[hemoglobina, okaloosa]",Sustancia,"[-0.660218, 0.112245, 0.067039, 0.441094, -0.50456, -0.724666, -0.107986, 0.313558, 0.198024, 1...."
1,102002,"[hb, 48, cd7, leu, arg]",Sustancia,"[-0.317944, 1.87942, 1.76004, 1.72051, 0.75724, -1.13105, -0.499457, -1.84984, -0.570444, -0.630..."
2,103007,"[virus, fibroma, ardillas]",Organismo,"[-0.14364, -0.006139, 0.080456, 0.936581, -1.16385, -0.552844, 0.081489, -0.584596, -0.185695, 0..."
3,104001,"[escision, lesion, rotula]",Procedimiento,"[-1.0588, -0.371483, -1.84789, 1.65618, -0.235278, 0.101576, -1.08105, 0.863104, -0.374193, 1.04..."
4,104001,"[escision, local, lesion, o, tejido, rotula]",Procedimiento,"[-0.178396, -0.865999, -2.62099, 3.59125, -1.68036, -0.115811, -1.95361, 0.583631, -1.50294, 1.5..."


### Proceso temporal para añadirle al Jerarquia, del otro DataFrame:

In [ ]:
# Carga de Datos para el Trabajo final:
fich = '/Snomed-CT/Diccionario/Snomed2vec-Jer-work.txt'
f = p+fich

dfSno = pd.read_csv(f, sep='|', encoding='utf_8', header=0, dtype='object').fillna('')

In [ ]:
dfSno.head()

In [ ]:
cols=['idConcept','jerarquia']
dfSnoFinal = pd.merge(dfSno[cols], SnomedWork, on='idConcept')

In [ ]:
dfSnoFinal.tail()

In [ ]:
# Creación de una columan nueva con el idConcepto+Terminos+Jerarquia
def unir(row):
    strTerm = ' '.join(row.corpusTerm)
    buff = row.idConcept + '|' + strTerm + '|' + row.jerarquia
    return buff

In [ ]:
# Creación de una columan nueva con el idConcepto+Terminos+Jerarquia
def unir2(row):
    #strTerm = ' '.join(row.Term)
    buff = row.idConcept + '|' + row.Term
    return buff

In [ ]:
# Creación de una columan nueva con el idConcepto+Terminos+Jerarquia
def unir3(row):
    strTerm = ' '.join(row.corpusTerm)
    buff = row.idConcept + '|' + strTerm+'| '+row.jerarquia
    return buff

In [ ]:
#SnomedWork['concept'] = SnomedWork[['idConcept', 'corpusTerm','jerarquia']].apply(unir, axis=1)
#SnomedWork['concept'] = SnomedWork[['idConcept','Term']].apply(unir2, axis=1)
SnomedWork['concept'] = SnomedWork[['idConcept','jerarquia','corpusTerm']].apply(unir3, axis=1)

In [ ]:
print('Comprobacion de tamaño Modelo Espacio Vectorial (1)- 516211 - %d' % len(SnomedWork))

## 6. Métodos Snomed2Vec.Most_similar() y Snomed2Vec.Similarity.

### 6.1 Pruebas del método Snomed2vec.Most_similar():
Implementacion método Most_Similar para Snomed2Vec. Basado en la implementación de gensim Word2Vec.

In [ ]:
def init_sims(self, replace=False):
        """
        Precompute L2-normalized vectors.
        If `replace` is set, forget the original vectors and only keep the normalized
        ones = saves lots of memory!
        Note that you **cannot continue training** after doing a replace. The model becomes
        effectively read-only = you can call `most_similar`, `similarity` etc., but not `train`.
        """
        if getattr(self, 'syn0norm', None) is None or replace:
            logger.info("precomputing L2-norms of word weight vectors")
            if replace:
                for i in xrange(w2v_InfAltaSK.syn0.shape[0]):
                    w2v_InfAltaSK.syn0[i, :] /= sqrt((w2v_InfAltaSK.syn0[i, :] ** 2).sum(-1))
                w2v_InfAltaSK.syn0norm = w2v_InfAltaSK.syn0
            else:
                w2v_InfAltaSK.syn0norm = (w2v_InfAltaSK.syn0 / sqrt((w2v_InfAltaSK.syn0 ** 2).sum(-1))[..., newaxis]).astype(REAL)

In [ ]:
self.vocab = {}  # mapping from a word (string) to a Vocab object
self.index2word = []  # map from a word's matrix index (int) to word (string)

self.vocab[word].index
syn0norm[self.vocab[word].index]

In [ ]:
w2v_InfAltaSK.vocab

In [11]:
len(w2v_InfAltaSK.index2word)

98103

In [14]:
w2v_InfAltaSK.index2word.index('infarto')

4578

In [15]:
word='infarto'
w2v_InfAltaSK.vocab[word].index

4578

In [20]:
w2v_InfAltaSK.syn0norm

In [ ]:
word='infarto'
w2v_InfAltaSK.syn0norm[w2v_InfAltaSK.vocab[word].index]

In [26]:
word='infarto'
positive=[word]
positive = [(word, 1.0) if isinstance(word, string_types + (ndarray,)) else word for word in positive]
print(positive)

[('infarto', 1.0)]


In [27]:
all_words, mean = set(), []

In [ ]:
for word, weight in positive:
    if isinstance(word, ndarray):
        mean.append(weight * word)
    elif word in w2v_InfAltaSK.vocab:
        mean.append(weight * w2v_InfAltaSK.syn0norm[w2v_InfAltaSK.vocab[word].index])
        all_words.add(w2v_InfAltaSK.vocab[word].index)
    else:
        raise KeyError("word '%s' not in vocabulary" % word)
    if not mean:
        raise ValueError("cannot compute similarity with no input")
    mean = matutils.unitvec(array(mean).mean(axis=0)).astype(REAL)

In [ ]:
if indexer is not None:
    indexer.most_similar(mean, 10)

In [ ]:
# Version Original Word2Vec:

def most_similar(self, positive=[], negative=[], topn=10, restrict_vocab=None, indexer=None):

        self.init_sims()

        if isinstance(positive, string_types) and not negative:
            # allow calls like most_similar('dog'), as a shorthand for most_similar(['dog'])
            positive = [positive]

        # add weights for each word, if not already present; default to 1.0 for positive and -1.0 for negative words
        positive = [
            (word, 1.0) if isinstance(word, string_types + (ndarray,)) else word
            for word in positive
        ]
        negative = [
            (word, -1.0) if isinstance(word, string_types + (ndarray,)) else word
            for word in negative
        ]

        # compute the weighted average of all words
        all_words, mean = set(), []
        for word, weight in positive + negative:
            if isinstance(word, ndarray):
                mean.append(weight * word)
            elif word in self.vocab:
                mean.append(weight * self.syn0norm[self.vocab[word].index])
                all_words.add(self.vocab[word].index)
            else:
                raise KeyError("word '%s' not in vocabulary" % word)
        if not mean:
            raise ValueError("cannot compute similarity with no input")
        mean = matutils.unitvec(array(mean).mean(axis=0)).astype(REAL)

        if indexer is not None:
            return indexer.most_similar(mean, topn)

        limited = self.syn0norm if restrict_vocab is None else self.syn0norm[:restrict_vocab]
        dists = dot(limited, mean)
        if not topn:
            return dists
        best = matutils.argsort(dists, topn=topn + len(all_words), reverse=True)
        # ignore (don't return) words from the input
        result = [(self.index2word[sim], float(dists[sim])) for sim in best if sim not in all_words]
        return result[:topn]

In [ ]:
# Funcion similitud Distancia del Coseno, Parametros Vectores
def vSimilar(wv1,wv2):
    return np.nan_to_num((round(float(np.dot(wv1, wv2)/(np.linalg.norm(wv1) * np.linalg.norm(wv2))),4)))

## 7. Sistema preparacion de la generación vectores para la gestion de las querys

### 7.1. Generación Vectores:
Generar los vectores, según los tokens de la frase del informe y de los conceptos candidatos del Snomed-CT. Teniendo en cuenta los 1-gramas, 2-gramas, 3-gramas. siendo el vector final de una frase, la suma de los vectores de los tokens que lo componen.<br>
**Funciones Auxiliares generadas:**
* **ngrams(listTokens, n):**. 
    * Entrada: **listTokens**, (lista de Tokens), **n**: número de n-gramas que genera.
    * Salida: **buffer**, Lista de n-gramas, según el valor de n, genera 
* **todosGramas(listTokens,n):**
    * Entrada: **listTokens**, (lista de Tokens), **n**: número de n-gramas máximo que genera, 1-gram,2-gram, 3-gram.
    * Salida: **buffer**, Lista de n-gramas, según el valor de n, genera n-gramas, desde 1-grama hasta n-gramas.

### 7.2. Generación de n-gramas según la frase compuesta de tokens.

In [7]:
# Calcula los n-gramas, segun una lista de tokens, valor n
def ngrams(listTokens, n):
    output = []  
    for i in range(len(listTokens)-n+1):
        output.append(listTokens[i:i+n])
    return output

In [8]:
# Crea todos los n-gramas de una frase según (n)
#Salida: lista de n-gramas desde 1 hasta n
def todosGramas(listTokens,n):
    output=[]
    buffer=[]
    for i in range(n):
        i+=1
        output.append(ngrams(listTokens, i))
    for i in range(n):
        buffer+= output[i]
    return buffer

In [9]:
# Entrada: Lista de gramas ['1-grama','2-grama',['3-grama','4-grama]]
from nltk.util import skipgrams

def skipgramas(frase,n,k):
    return(skipgrams(sent, n, k))

### 7.4. Crear Vector para cada token de la frase.
Generación de los vectores de los tokens de cada frase.<br>
* Funcion **creaVector(frase, size)**, genera un vector por cada token de la frase.
    * Entrada: lista de tokens de la **frase** y **size**, tamaño del vector generado según modelo W2V (300 o 100)
    * Salida: lista con los vectores de cada token, de la frase

* Funcion: **vectorTotal(listGramas, size)**
    * Entrada: **listGramas**, lista de n-gramas generados, **size**, tamaño del vector, según modelo W2V.
    * Salida: Vectores asociados a cada frase, si es 1-grama -> Salida Vector(token).<br>
      Si la frase tiene **n** gramas -> Salida $$(V[p]=v[tk_{1}]+v[tk_{2}]+...+v[tk_{n}])$$

### 7.5. Crear los vectores para todos los ngramas generados:

In [71]:
# Entrada: lista de Frase, con ngramas en sublistas
# Ejemplo: todosGramas(textP13,3) 
# Entrada: ['fiebre','gripe','tos']
# Salida: [['fiebre'],['gripe'],['tos'],['fiebre', 'gripe'], ['gripe', 'tos'],['fiebre', 'gripe', 'tos']]

def vectorTotal(lstgrams,size):
    output=[]
    for item in lstgrams:
        buffer=np.zeros((size,), dtype=np.float32)
        for w in item:
            try:
                buffer = buffer + w2v_InfAltaSK.word_vec(w)
            except:
                # Si el token no está en el vocabulario del Modelo, se descarta todos los ngramas
                buffer = buffer + np.zeros((size,), dtype=np.float32)
                #break
        output.append((item, buffer))
    return output

### 6.5. Crear los vectores asociados para las descripciones de los conceptos de Snomed-CT.
Tenemos dos opciones:
1. Según el subconjunto seleccionado de los conceptos,con algún token de la frase del informe, nombrado en la descripción del concepto y sus sinónimos.
1. Generar el Corpus final, con los vectores de todos los sinónimos de Snomed-CT. para realizar luego lso cálculos mas rápido.

### 6.6. **Cáculo del Vector** de los sinónimos de Snomed-CT de los **Conceptos candidatos:**

Función: **vsynSnomed(candidatos, sizeVector):**
   * Entrada: **candidatos**, lista con los conceptos candidatos de Snomed-CT, **sizeVector** Tamaño W2V size.
   * Salida: El vector asociado para cada descripción de Snomed-CT, según el concepto.

In [11]:
def vsynSnomed(candidatos, sizeVector):
    output=[]
    for i in range(len(candidatos)):
        codIdSnomd = candidatos[i][0]
        SynsSnomd=candidatos[i][2]
        output.append((codIdSnomd,vectorTotal(SynsSnomd, sizeVector)) )
    return output

### 6.7. Creación vectores finales todo el Snomed-CT.
Se genera un vector para cada descripción de cada concepto y se genera la matriz de conceptos.

In [ ]:
def creaVecSnomed(row):
    output=[]
    desc=row.corpus_Syn
    cod = row.conceptId
    for desc_syn in range(len(desc)):
        output.append((cod,vectorTotal(desc_syn, sizeVector)) )
    return

In [14]:
cols=['conceptId','corpus_Syn']
dfSnomedSyn[cols].head()

,conceptId,corpus_Syn
0,100000000,"[[bitter, 3]]"
1,10000006,"[[dolor, pecho, irradiado, hacia, zonas]]"
2,1000004,"[[esguince], [estrema], [lesion, articular], [torcedura], [traumatismo, articular]]"
3,100009004,"[[bloat, pac]]"
4,10001005,"[[sepsis, bacteriana], [septicemia, bacteriana]]"


** Enfoques para la estructura final de Snomed2vec:**
* Generar estructura Matriz con una linea por cada ID (Id único), con los vectores de sus descripciones y sinónimos en misma fila.
* Generar una linea por cada id y su descripcion. (Id's repetidos).

In [26]:
# Generar estructura Matriz con una linea por cada ID, con los vectores de sus descripciones y sinónimos 
# en misma fila.
# ----------------------------------------------------
# Enfoque 1:

snomed2vec = []

for i in range(len(dfSnomedSyn)):
    codId = dfSnomedSyn['conceptId'].loc[i]
    txtCorpus = dfSnomedSyn['corpus_token'].loc[i]
    txtSyn = dfSnomedSyn['corpus_Syn'].loc[i]
    snomed2vec.append([codId,vectorTotal(txtSyn, 300)])
    
print(len(snomed2vec))

In [46]:
txtSyn = dfSnomedSyn['corpus_Syn'].loc[2]
txtSyn

[['esguince'],
 ['estrema'],
 ['lesion', 'articular'],
 ['torcedura'],
 ['traumatismo', 'articular']]

In [27]:
def vectorSnomed(lstgrams,size):
    output=[]
    buffer=np.zeros((size,), dtype=np.float32)
    for item in lstgrams:
        try:
            buffer = buffer + w2v_InfAltaSK.word_vec(item)
        except:
            buffer = buffer + np.zeros((size,), dtype=np.float32)
            #break
    output.extend( buffer)
    return output

In [28]:
# Generar una linea por cada id y su descripcion. (Id's repetidos).
# Enfoque 2 Version 1.0:

snomed2vec = []

for i in range(len(dfSnomedSyn)):
    codId = dfSnomedSyn['conceptId'].loc[i]
    txtCorpus = dfSnomedSyn['corpus_token'].loc[i]
    txtSyn = dfSnomedSyn['corpus_Syn'].loc[i]
    for txt1 in txtSyn:
        snomed2vec.extend([codId,txt1, vectorSnomed(txt1, 300)])
        
print(len(snomed2vec))

1432986


In [56]:
# Generar una linea por cada id y su descripcion. (Id's repetidos).
# Enfoque 2 Version 1.1:

snomed2vec = []

for i in range(len(dfSnomedSyn)):
    codId = dfSnomedSyn['conceptId'].loc[i]
    txtCorpus = dfSnomedSyn['corpus_token'].loc[i]
    txtSyn = dfSnomedSyn['corpus_Syn'].loc[i]
    for txt1 in txtSyn:
        concept = codId+'|'+' '.join(txt1)
        snomed2vec.extend([concept, vectorSnomed(txt1, 300)])
        
print(len(snomed2vec))

955324


In [8]:
txtSyn = dfSnomedSyn['corpus_Syn'].loc[2]
txtSyn

[['esguince'],
 ['estrema'],
 ['lesion', 'articular'],
 ['torcedura'],
 ['traumatismo', 'articular']]

In [58]:
snomed2vec[0:18:2]

['100000000|bitter 3',
 '10000006|dolor pecho irradiado hacia zonas',
 '1000004|esguince',
 '1000004|estrema',
 '1000004|lesion articular',
 '1000004|torcedura',
 '1000004|traumatismo articular',
 '100009004|bloat pac',
 '10001005|sepsis bacteriana']

In [40]:
snomed2vec[1:24:3]

[['bitter', '3'],
 ['dolor', 'pecho', 'irradiado', 'hacia', 'zonas'],
 ['esguince'],
 ['estrema'],
 ['lesion', 'articular'],
 ['torcedura'],
 ['traumatismo', 'articular'],
 ['bloat', 'pac']]

In [ ]:
w2v_InfAltaSK.most_similar()

### 6.8. Convertir la lista generada un DataFrame Pandas (dfSnomed2vec):

In [ ]:
# Cración de un TensorTensorflow Project, para visualizar el resultado final:

fich = '/Models/Snomed2vec.tsv'
f = p + fich

tensor_file = '{0}_tensor.tsv'.format(output)
metadata_file = '{0}_metadata.tsv'.format(output)
with open(tensor_file, 'w+') as file_vector:
    with open(metadata_file, 'w+') as file_metadata:
        for word in model.wv.index2word:
            try:
                w = model.wv.vocab[word]
                file_metadata.write(word + '\t' + str(w.count) + '\n')
                vector_row = '\t'.join(str(x) for x in model[word])
                file_vector.write(vector_row + '\n')
            except Exception:
                logging.warning('Invalid characters in word: {0}'.format(str(word)))

In [ ]:
f = open ('/media/nacho/DatosHD/Doctorado/codigo/DataFinal/BagOfConcepts.txt','w')
for elemento in BagOfConcepts:
    f.write('%s %s\n' % elemento)
f.close()

In [ ]:
import numpy as np
x = [['ABC', 123.45], ['DEF', 678.90]]
np.savetxt('text.txt', x, fmt='%s')
 
# default dtype for  np.loadtxt is also floating point, change it, to be able to load mixed data.
y = np.loadtxt('text.txt', dtype=np.object)

In [ ]:
fich = '/Models/Snomed2vec.txt'
f = p + fich
f = open (f,'wb')
for i in range(0, len(snomed2vec), 2):
    buffer = snomed2vec[i]+'\t'+snomed2vec[i+1]
    f.write('%s \n' % buffer)
f.close()

In [62]:
lista_final = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for i in range(0, 16, 2):
    lista_final[i] = i
    lista_final[i+1] = i
print(lista_final)


#npSnomed2Vec = np.array(snomed2vec).reshape(len(snomed2vec),2)

In [61]:
#dfsnomed2vec = pd.DataFrame(np.array(snomed2vec).reshape(len(snomed2vec),2), columns = ['idConcept','vector'])

In [ ]:
dfsnomed2vec.head()

In [30]:
#snomed2vec[5][1]

### 7.0 Función de Similitud. (Mtodo de la nueva clase):
Función de Similitud, cuando se le pasa los vectores generados. Utilizando la Distancia del Coseno.

In [42]:
# Funcion similitud Distancia del Coseno, Parametros Vectores
def vSimilar(wv1,wv2):
    return np.nan_to_num((round(float(np.dot(wv1, wv2)/(np.linalg.norm(wv1) * np.linalg.norm(wv2))),4)))

In [ ]:
def levenshtein(first, second):
    """Return the Levenshtein distance between two strings.
    Based on:
        http://rosettacode.org/wiki/Levenshtein_distance#Python
        implementing at: https://github.com/coloratto/TextRank/blob/master/textrank/__init__.py
    """
    if len(first) > len(second):
        first, second = second, first
    distances = range(len(first) + 1)
    for index2, char2 in enumerate(second):
        new_distances = [index2 + 1]
        for index1, char1 in enumerate(first):
            if char1 == char2:
                new_distances.append(distances[index1])
            else:
                new_distances.append(1 + min((distances[index1],
                                             distances[index1 + 1],
                                             new_distances[-1])))
        distances = new_distances
    return distances[-1]

In [43]:
def masCercana(w):
    cerca=[]
    try:
        cerca=w2v_InfAltaSK.most_similar(w)
    except:
        cerca=[]
    return cerca

In [44]:
def cercanos(sent,n):
    simConcept=[]
    for w in sent:
        buffer = masCercana(w)
        if len(masCercana(w))>1:
            for i in range(n):
                simConcept.append(buffer[i][0])
    return simConcept

### Pruebas para entender funcionamiento de most_similar()

In [3]:
InfAltModel.syn0.shape

(98103, 300)

In [8]:
InfAltModel.syn0.shape[0]

98103

In [9]:
InfAltModel.syn0.shape[1]

300

In [8]:
InfAltModel.syn0norm

In [113]:
InfAltModel.syn0norm = (InfAltModel.syn0 / sqrt((InfAltModel.syn0 ** 2).sum(-1))[..., newaxis]).astype(REAL)

In [114]:
InfAltModel.syn0norm.shape

(98103, 300)

In [115]:
positive=['dlp']

In [116]:
positive = [(word, 1.0) if isinstance(word, string_types + (ndarray,)) else word for word in positive]

In [117]:
positive

[('dlp', 1.0)]

In [118]:
all_words, mean = set(), []

In [121]:
ind = InfAltModel.vocab[word].index
vecWord=InfAltModel.syn0norm[ind]
#vecWord

In [123]:
for word, weight in positive:
    if isinstance(word, ndarray):
        mean.append(weight * word)
    elif word in InfAltModel.vocab:
        mean.append(weight * InfAltModel.syn0norm[InfAltModel.vocab[word].index])
        all_words.add(InfAltModel.vocab[word].index)
    else:
        raise KeyError("word '%s' not in vocabulary" % word)
    if not mean:
        raise ValueError("cannot compute similarity with no input")
    mean = matutils.unitvec(array(mean).mean(axis=0)).astype(REAL)

In [124]:
all_words

{14311}

In [130]:
mean.shape

(300,)

In [109]:
#mean

In [92]:
limited = InfAltModel.syn0norm
limited.shape

(98103, 300)

In [93]:
dists = dot(limited, mean)

In [94]:
dists.shape

(98103,)

In [111]:
len(dists)

98103

In [95]:
len(all_words)

1

In [96]:
best = matutils.argsort(dists, topn=10 + len(all_words), reverse=True)

In [97]:
best

array([46071, 68186, 49885, 70627, 68051, 64485, 71493, 66225, 53562,
       44410, 73181])

In [98]:
result = [(InfAltModel.index2word[sim], float(dists[sim])) for sim in best if sim not in all_words]

In [132]:
result

[('puntualres', 0.8591750264167786),
 ('hemtemesis', 0.8501303195953369),
 ('consiuderamos', 0.846984326839447),
 ('presentandon', 0.8460383415222168),
 ('odinamicamemte', 0.8438538312911987),
 ('presenanto', 0.8424177765846252),
 ('transitorios', 0.8421307802200317),
 ('hemodinammicamente', 0.8367821574211121),
 ('hemodinamicamene', 0.8348478078842163),
 ('mostar', 0.8338656425476074)]

## Generación de un Modelo W2V-Snomed, de tamaño del vector 300. (Word2Vec)
Se elige este tamaño, para que coincida, con los vectores creados en el Modelo de los Informes de Alta.

In [ ]:
from gensim.models import Word2Vec

model300 = Word2Vec(dfSnomedSyn.corpus_token,
                size=300,
                window=3,
                workers=4,
                min_count=1,
                sample=0.05,
                sg=1,
                iter=5,
                hs=0)

In [4]:
# Grabamos el modelo generado al disco.
fich = '/Models/w2vSnomed-SK-s300-w3.txt'
f = p+fich
model300.wv.save_word2vec_format(f, binary=False)

## Generación de un Modelo D2V-Snomed, de tamaño del vector 300. (Doc2Vec)
Se elige este tamaño, para que coincida, con los vectores creados en el Modelo de los Informes de Alta.

## Fase 2. Creación Modelo Doc2Vec. -----------------------------------------------------------------------
Para generar el modelo Doc2Vec, se debe elegir que tipo d reprensetación de palabras vamos a querer, y cual va ser la finalidad.<br>
Existen dos tipos de modelos:<br>
1. **PV-DM: dm=1**
1. **PV-DBOW: dm=0**

In [67]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [68]:
data =dfSnomedSyn['corpus'].tolist()

In [69]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

In [76]:
tagged_data[:3]

[TaggedDocument(words=['bitter', '3'], tags=['0']),
 TaggedDocument(words=['dolor', 'de', 'pecho', 'irradiado', 'hacia', 'otras', 'zonas'], tags=['1']),
 TaggedDocument(words=['esguince', '|', 'estrema', '|', 'lesion', 'articular', '|', 'torcedura', '|', 'traumatismo', 'articular'], tags=['2'])]

### Crear tagSnomed Para identificar los conceptos de Snomed:

Enfoque **1**.:

In [112]:
cols=['corpus','conceptId']
#data =dfSnomedSyn[cols].tolist()
dataCorpus =dfSnomedSyn['corpus'].tolist()
dataId =dfSnomedSyn['conceptId'].tolist()
dataToken =dfSnomedSyn['corpus_token'].tolist()

In [114]:
dataId[2]+'|'+' '.join(dataToken[2])

'1000004|traumatismo estrema articular esguince lesion torcedura'

In [123]:
tagged_data = [TaggedDocument(words=dataToken[i], tags=dataId[i]+'|'+dataCorpus[i]) for i, _d in enumerate(dataId)]

In [ ]:
tagged_data

Enfoque **2**.: 

**Crear nuevo Corpus, poniendo en una linea, el ID y su descripción:**

In [92]:
dataPreTag=[]
for i in range(len(dataId)):
    cod  = dataId[i]
    desc = dataCorpus[i].split('|')
    for item in desc:
        dataPreTag.append([cod,item])
len(dataPreTag)

477662

In [93]:
dataPreTag[0:16]

[['100000000', 'bitter   3'],
 ['10000006', 'dolor de pecho irradiado hacia otras zonas'],
 ['1000004', 'esguince '],
 ['1000004', ' estrema '],
 ['1000004', ' lesion articular '],
 ['1000004', ' torcedura '],
 ['1000004', ' traumatismo articular'],
 ['100009004', 'bloat   pac'],
 ['10001005', 'sepsis bacteriana '],
 ['10001005', ' septicemia bacteriana'],
 ['100019005', 'bo   bac   2x'],
 ['10002003', 'fundectomia gastrica '],
 ['10002003', ' reseccion de la cupula del estomago '],
 ['10002003', ' reseccion del fondo gastrico'],
 ['100023002', 'borditech   p e '],
 ['100024008', 'bo   se']]

In [95]:
dataPreTag[0][1]

'bitter   3'

In [106]:
tagged_data = [TaggedDocument(words=word_tokenize(dataPreTag[i][1].lower()), tags=[dataPreTag[i][0]+'-'+dataPreTag[i][1]]) for i, _d in enumerate(dataPreTag)]

In [ ]:
tagged_data

### Crear Diccionario look-up para identificar a los Id's de los Conceptos Snomed

### Entrenar el Modelo Doc2Vec (PV-DBOW):

In [ ]:
max_epochs = 20
vec_size = 300
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=2,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

In [8]:
fich = '/models/Doc2Vec-Snomed-PV-DBOW-s300'
f = p+fich
model.save(f)
print("Model Saved")

### Cargar el Modelo ya generado desde disco:

In [ ]:
fich = '/models/Doc2Vec-Snomed-PV-DBOW-s300'
f = p+fich
model = gensim.models.doc2vec.Doc2Vec.load(f)

### Aplicar Most_Similar documentos:

In [130]:
token_list = ['esguince','tobillo']
ivec = model.infer_vector(doc_words=token_list, steps=20, alpha=0.025)
print(model.docvecs.most_similar(positive=[ivec], topn=10))

[('|', 0.1533329337835312), (':', 0.15107718110084534), ('v', 0.1258426159620285), ('d', 0.1036207526922226), ('g', 0.098814956843853), ('x', 0.07565678656101227), ('m', 0.07097321003675461), ('ø', 0.06962880492210388), ('z', 0.06912010908126831), ('º', 0.06781326234340668)]


In [127]:
token_list = ['esguince','tobillo']
ivec = model.infer_vector(doc_words=token_list, steps=20, alpha=0.025)
print(model.most_similar(positive=[ivec], topn=10))

2018-08-18 13:38:08,892 : INFO : precomputing L2-norms of word weight vectors


[('oseas', 0.6812828183174133), ('psoas', 0.6541339159011841), ('astragaloescafoidea', 0.6537420749664307), ('senos', 0.6340361833572388), ('eneldo', 0.6295666694641113), ('escalenos', 0.6285560131072998), ('ahusados', 0.6274730563163757), ('pantallas', 0.6255570650100708), ('antiesquistosomiasico', 0.6173219680786133), ('nelore', 0.6170296669006348)]


In [128]:
txt='esguince'
print(model.most_similar(positive=[txt], topn=10))

[('sinoviectomia', 0.8291350603103638), ('sinovectomia', 0.8246129751205444), ('artroscopia', 0.7749430537223816), ('suprastragalina', 0.764326274394989), ('menisco', 0.7619028687477112), ('glenohumeral', 0.7592365741729736), ('cubitohumeral', 0.7559356689453125), ('anquilosis', 0.7460286617279053), ('fusion', 0.7286036014556885), ('escafosemilunar', 0.725143551826477)]


In [129]:
#txt=['esguince','tobillo']
#print(model.docvecs.most_similar(positive=[txt], topn=10))

In [13]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(tagged_data[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(tagged_data[sims[index][0]].words)))

### 6.0 Pruebas de Ejecución:
Se prueba con varias frases, para identificar el funcionamiento del algoritmo.<br>
Se calcula Paso 1 y Paso 2..

### 7.0 Guardar los resultados en disco:

In [41]:
# Guardar los datos para comprobacion posteriori
import csv
fich = '/DataWork/CorpusGold/boc-Gold-AnatoPato.txt'
#fich = '/DataWork/CorpusGold/boc-CorpusGold.txt'
fich = p+fich

with open(fich, "w") as fichero:
    writer = csv.writer(fichero)
    writer.writerows(BoC)